In [1]:
import os
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [2]:
def load_spectrum_file(dir_path):
    data = []
    label = []
    file_dir_list = os.listdir(dir_path)
    for file_dir in file_dir_list:
        file_list = os.listdir(dir_path + '/' + file_dir)
        for filename in file_list:
            file_path = dir_path + '/' + file_dir + '/' + filename
            x, y = np.loadtxt(file_path, dtype=float, comments='#', delimiter=',', unpack=True)
            data.append(y)
            label.append(file_dir)
    return data, label

In [14]:
print('Loading data...')
dir_path = '../animal_blood/animal_blood_preprocessing_classify'
# dir_path = '../Dataset_1'
data, label = load_spectrum_file(dir_path)
data = np.array(data)
le = LabelEncoder()
labels = le.fit_transform(label)
train_data, test_data, train_labels, test_labels = train_test_split(data, labels)
print('Completed!')

Loading data...
Completed!


In [15]:
# 硬投票分类器预测
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

log_clf = LogisticRegression()
rnd_clf = RandomForestClassifier()
svm_clf = SVC()

voting_clf = VotingClassifier(
    estimators=[('lr', log_clf), ('rf', rnd_clf), ('svc', svm_clf)]
    , voting = 'hard'
)
voting_clf.fit(train_data, train_labels)

/Users/renpengju/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


VotingClassifier(estimators=[('lr', LogisticRegression()),
                             ('rf', RandomForestClassifier()), ('svc', SVC())])

In [16]:
# 测试每个分类器精度
from sklearn.metrics import accuracy_score
for clf in (log_clf, rnd_clf, svm_clf, voting_clf):
    clf.fit(train_data, train_labels)
    y_pred = clf.predict(test_data)
    print(clf.__class__.__name__, accuracy_score(test_labels, y_pred))
    
# VOtingClassifier
# animal_blood_orginal_classify 0.8915
# animal_blood_preprocessing_classify 1.0

/Users/renpengju/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression 1.0
RandomForestClassifier 0.963855421686747
SVC 0.7831325301204819


/Users/renpengju/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


VotingClassifier 1.0


In [17]:
train_labels.shape

(247,)

In [18]:
# bagging and pasting
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

bag_clf = BaggingClassifier(
    DecisionTreeClassifier()
    , n_estimators=500
    , max_samples=247
    , bootstrap=True
    , n_jobs=-1
)
bag_clf.fit(train_data, train_labels)
y_pred = bag_clf.predict(test_data)

In [19]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(bag_clf, data, labels, cv=5)
print('scores:', scores)

scores: [0.96969697 0.93939394 0.96969697 0.93939394 0.95454545]


In [20]:
print(bag_clf.__class__.__name__, accuracy_score(test_labels, y_pred))

# animal_blood_orginal_classify 0.7710
# animal_blood_preprocessing_classify 0.9277

BaggingClassifier 0.927710843373494


In [13]:
train_data.shape

(247, 637)